# Chapter 22: DAOs (Decentralized Autonomous Organizations)

---

Decentralized Autonomous Organizations (DAOs) represent a paradigm shift in how groups coordinate and make decisions. They are internet-native organizations collectively owned and managed by their members, with rules encoded in smart contracts. DAOs have no central leadership; decisions are made via proposals and voting, and treasury funds are controlled by code. In this chapter, we'll explore what DAOs are, how they govern themselves, and how to build one using modern tools and frameworks.

---

## 22.1 Introduction to DAOs

### 22.1.1 What is a DAO?

A DAO is an organization represented by rules encoded as a computer program that is transparent, controlled by organization members, and not influenced by a central government. In simpler terms, a DAO is a group of people who share a common goal, pool resources in a treasury, and make decisions through collective voting, all enforced by smart contracts on a blockchain.

**Key characteristics:**
- **Decentralized**: No single entity has absolute control; power is distributed among members.
- **Autonomous**: Rules are enforced by code, not by people. The organization can operate without manual intervention.
- **Transparent**: All proposals, votes, and treasury transactions are recorded on-chain and publicly visible.
- **Member-owned**: Typically, members hold tokens that represent voting power and ownership.

**Analogy:** A DAO is like a co-op or a partnership, but instead of paper agreements and meetings, it uses smart contracts and online voting.

### 22.1.2 Governance Without Central Authority

Traditional organizations have a hierarchical structure: CEO, board, managers. DAOs invert this—power flows from the bottom up. Members propose changes, vote on them, and if approved, the code executes them automatically.

**How it works:**
1. **Proposal**: Any member (or members with sufficient token weight) can create a proposal—e.g., "Transfer 10 ETH to Alice for marketing services."
2. **Voting**: Members vote using their governance tokens. Voting may last several days.
3. **Execution**: If the proposal passes (e.g., achieves quorum and majority), the associated smart contract action (like transferring funds) is executed, often after a timelock delay.

This process eliminates the need for trust in leaders—everyone can verify the rules and outcomes.

### 22.1.3 Notable DAO Examples

| DAO | Description |
|-----|-------------|
| **MakerDAO** | Governs the DAI stablecoin. MKR token holders vote on risk parameters, collateral types, and fees. |
| **Uniswap** | Governs the Uniswap protocol. UNI holders vote on fee switches, treasury allocations, and protocol upgrades. |
| **Compound** | COMP token holders govern the lending protocol's interest rate models and asset listings. |
| **ENS DAO** | Manages the Ethereum Name Service. ENS holders vote on pricing, upgrades, and ecosystem grants. |
| **Aragon** | A platform for creating and managing DAOs. Its own DAO governs the Aragon project. |
| **ConstitutionDAO** | A famous (though short-lived) DAO that raised over $40M to buy an original copy of the US Constitution. |

These examples illustrate the diversity of DAOs—from protocol governance to social coordination.

---

## 22.2 DAO Governance Mechanisms

DAOs need robust mechanisms to translate member preferences into decisions. The most common is token-based voting, but variations exist.

### 22.2.1 Token-Based Voting

In token-based voting, each governance token represents one vote (or weighted vote). This is simple but can lead to plutocracy—large token holders dominate.

**Implementation considerations:**
- **Quorum**: Minimum participation required for a proposal to be valid (e.g., 4% of total supply).
- **Voting period**: Duration (e.g., 3 days) during which votes can be cast.
- **Vote thresholds**: Often a simple majority (>50% of votes cast) is required.

**Example voting contract snippet:**
```solidity
contract SimpleVoting {
    struct Proposal {
        string description;
        uint256 forVotes;
        uint256 againstVotes;
        uint256 startBlock;
        uint256 endBlock;
        bool executed;
    }

    mapping(address => uint256) public votes;
    Proposal[] public proposals;

    function createProposal(string memory description, uint256 votingPeriod) external {
        proposals.push(Proposal({
            description: description,
            forVotes: 0,
            againstVotes: 0,
            startBlock: block.number,
            endBlock: block.number + votingPeriod,
            executed: false
        }));
    }

    function vote(uint256 proposalId, bool support) external {
        Proposal storage p = proposals[proposalId];
        require(block.number >= p.startBlock && block.number <= p.endBlock, "Voting not active");
        uint256 voterWeight = votes[msg.sender]; // assume tokens are locked/weighted
        if (support) {
            p.forVotes += voterWeight;
        } else {
            p.againstVotes += voterWeight;
        }
    }
}
```

### 22.2.2 Quadratic Voting

Quadratic voting aims to reduce plutocracy by making the cost of additional votes increase quadratically. If you have 100 tokens, one vote costs 1 token, but two votes cost 4 tokens, three cost 9, etc. This gives smaller holders more relative influence.

While powerful, quadratic voting is complex to implement and can be gamed with multiple wallets. Some DAOs experiment with it, but token-based linear voting remains the norm.

### 22.2.3 Delegation

Delegation allows token holders to assign their voting power to a trusted representative (delegate). This encourages participation—holders who lack time to research every proposal can delegate to experts.

**Delegation mechanics:**
- A token holder can delegate their voting power to another address.
- The delegate can then vote on proposals using the aggregated power.
- Delegation can be changed at any time (though often with a delay to prevent vote-buying attacks).

OpenZeppelin's `ERC20Votes` extension implements delegation and vote tracking.

### 22.2.4 Proposal Lifecycle

A typical proposal goes through several stages:

```
┌────────────┐    ┌────────────┐    ┌────────────┐    ┌────────────┐    ┌────────────┐
│   Draft    │───▶│  Pending   │───▶│   Active   │───▶│  Succeeded │───▶│  Executed  │
└────────────┘    └────────────┘    └────────────┘    └────────────┘    └────────────┘
       │                 │                 │                 │
       ▼                 ▼                 ▼                 ▼
   Discussion     Awaiting vote    Voting period    Awaiting execution
   (off-chain)    start            in progress      (timelock)
```

1. **Draft**: Idea is discussed off-chain (forum, Discord).
2. **Pending**: Proposal is submitted on-chain but voting hasn't started.
3. **Active**: Voting period is open; token holders cast votes.
4. **Succeeded**: Proposal met quorum and majority; waiting for timelock.
5. **Executed**: After timelock, the action (e.g., transfer) is performed.

If a proposal fails (doesn't meet quorum or majority), it becomes "Defeated."

---

## 22.3 Building a DAO

Let's build a simple DAO using OpenZeppelin's Governor contracts. These contracts are modular and follow industry standards.

### 22.3.1 Governance Token Design

First, we need a token that supports voting and delegation. OpenZeppelin provides `ERC20Votes` for this purpose.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/token/ERC20/ERC20.sol";
import "@openzeppelin/contracts/token/ERC20/extensions/ERC20Permit.sol";
import "@openzeppelin/contracts/token/ERC20/extensions/ERC20Votes.sol";

contract GovernanceToken is ERC20, ERC20Permit, ERC20Votes {
    constructor(uint256 initialSupply) ERC20("GovernanceToken", "GOV") ERC20Permit("GovernanceToken") {
        _mint(msg.sender, initialSupply);
    }

    // The following functions are overrides required by Solidity.
    function _afterTokenTransfer(address from, address to, uint256 amount)
        internal
        override(ERC20, ERC20Votes)
    {
        super._afterTokenTransfer(from, to, amount);
    }

    function _mint(address to, uint256 amount)
        internal
        override(ERC20, ERC20Votes)
    {
        super._mint(to, amount);
    }

    function _burn(address account, uint256 amount)
        internal
        override(ERC20, ERC20Votes)
    {
        super._burn(account, amount);
    }
}
```

This token uses `ERC20Votes` to track voting power at each block, enabling delegation and checkpoints.

### 22.3.2 Voting Contract Implementation

We'll use OpenZeppelin's `Governor` contract, which is highly customizable. It requires a voting token and a timelock.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/governance/Governor.sol";
import "@openzeppelin/contracts/governance/extensions/GovernorSettings.sol";
import "@openzeppelin/contracts/governance/extensions/GovernorCountingSimple.sol";
import "@openzeppelin/contracts/governance/extensions/GovernorVotes.sol";
import "@openzeppelin/contracts/governance/extensions/GovernorVotesQuorumFraction.sol";
import "@openzeppelin/contracts/governance/extensions/GovernorTimelockControl.sol";

contract MyGovernor is
    Governor,
    GovernorSettings,
    GovernorCountingSimple,
    GovernorVotes,
    GovernorVotesQuorumFraction,
    GovernorTimelockControl
{
    constructor(IVotes _token, TimelockController _timelock)
        Governor("MyGovernor")
        GovernorSettings(7200 /* 1 day in blocks ~ 7200 */, 50400 /* 1 week */, 0)
        GovernorVotes(_token)
        GovernorVotesQuorumFraction(4) // 4% quorum
        GovernorTimelockControl(_timelock)
    {}

    function votingDelay() public view override(IGovernor, GovernorSettings) returns (uint256) {
        return super.votingDelay();
    }

    function votingPeriod() public view override(IGovernor, GovernorSettings) returns (uint256) {
        return super.votingPeriod();
    }

    function quorum(uint256 blockNumber)
        public
        view
        override(IGovernor, GovernorVotesQuorumFraction)
        returns (uint256)
    {
        return super.quorum(blockNumber);
    }

    function proposalThreshold()
        public
        view
        override(Governor, GovernorSettings)
        returns (uint256)
    {
        return super.proposalThreshold();
    }

    // The following functions are overrides required by Solidity.
    function state(uint256 proposalId)
        public
        view
        override(Governor, GovernorTimelockControl)
        returns (ProposalState)
    {
        return super.state(proposalId);
    }

    function _execute(uint256 proposalId, address[] memory targets, uint256[] memory values, bytes[] memory calldatas, bytes32 descriptionHash)
        internal
        override(Governor, GovernorTimelockControl)
    {
        super._execute(proposalId, targets, values, calldatas, descriptionHash);
    }

    function _cancel(address[] memory targets, uint256[] memory values, bytes[] memory calldatas, bytes32 descriptionHash)
        internal
        override(Governor, GovernorTimelockControl)
        returns (uint256)
    {
        return super._cancel(targets, values, calldatas, descriptionHash);
    }

    function _executor()
        internal
        view
        override(Governor, GovernorTimelockControl)
        returns (address)
    {
        return super._executor();
    }
}
```

This governor:
- Has a 1-day voting delay and 1-week voting period.
- Uses simple majority counting.
- Requires 4% quorum.
- Integrates with a timelock for delayed execution.

### 22.3.3 Treasury Management

The treasury is typically a smart contract that holds assets (ETH, ERC20s). It should be controlled by the DAO—only proposals that pass can move funds.

Often, the timelock contract itself acts as the treasury. The timelock holds funds and only releases them after a successful proposal and delay.

**TimelockController** from OpenZeppelin is the standard. It:
- Holds assets.
- Allows proposers (the governor) to schedule transactions.
- Enforces a minimum delay before execution.
- Allows cancellation (by cancelers, e.g., a multisig).

### 22.3.4 Timelock Controller

We need a timelock to add a safety delay between proposal passage and execution. This gives users time to exit if they disagree.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

import "@openzeppelin/contracts/governance/TimelockController.sol";

contract MyTimelock is TimelockController {
    constructor(
        uint256 minDelay,
        address[] memory proposers,
        address[] memory executors
    ) TimelockController(minDelay, proposers, executors, msg.sender) {}
}
```

- `minDelay`: Minimum delay in seconds (e.g., 2 days).
- `proposers`: Addresses allowed to schedule transactions (our governor).
- `executors`: Addresses allowed to execute after delay (can be same as proposers, or open).

**Deployment order:**
1. Deploy `GovernanceToken`.
2. Deploy `MyTimelock`, setting proposers to the governor address (not yet known) and executors to a broad set (e.g., everyone) or also the governor.
3. Deploy `MyGovernor`, passing the token and timelock addresses.
4. Transfer timelock admin rights to the governor (or a DAO-controlled multisig) so the governor can schedule.

**Deployment script (simplified):**
```javascript
const { ethers } = require("hardhat");

async function main() {
  // Deploy token
  const token = await ethers.deployContract("GovernanceToken", [ethers.parseEther("1000000")]);
  await token.waitForDeployment();
  console.log("Token deployed to:", token.target);

  // Deploy timelock with minDelay 2 days, proposers empty for now, executors any
  const minDelay = 2 * 24 * 60 * 60; // 2 days
  const proposers = []; // will be updated after governor deploy
  const executors = [ethers.ZeroAddress]; // anyone can execute
  const timelock = await ethers.deployContract("MyTimelock", [minDelay, proposers, executors]);
  await timelock.waitForDeployment();
  console.log("Timelock deployed to:", timelock.target);

  // Deploy governor
  const governor = await ethers.deployContract("MyGovernor", [token.target, timelock.target]);
  await governor.waitForDeployment();
  console.log("Governor deployed to:", governor.target);

  // Grant proposer role to governor in timelock
  const PROPOSER_ROLE = await timelock.PROPOSER_ROLE();
  await timelock.grantRole(PROPOSER_ROLE, governor.target);
  console.log("Governor granted proposer role");

  // Grant executor role to everyone (or to governor)
  const EXECUTOR_ROLE = await timelock.EXECUTOR_ROLE();
  await timelock.grantRole(EXECUTOR_ROLE, ethers.ZeroAddress); // allow anyone

  // Optionally, revoke admin role from deployer after setup
  const adminRole = await timelock.DEFAULT_ADMIN_ROLE();
  await timelock.revokeRole(adminRole, (await ethers.getSigners())[0].address);
}
```

Now the DAO is ready: token holders can create proposals via the governor, which schedules them in the timelock, and after the delay they can be executed.

---

## 22.4 DAO Tools and Frameworks

Building a DAO from scratch is educational, but in production you'll often use existing platforms that abstract away much of the complexity.

### 22.4.1 Aragon

[Aragon](https://aragon.org/) is a platform for creating and managing DAOs. It provides:
- **Aragon App**: A no-code interface to create a DAO, set up voting, and manage treasury.
- **Aragon OSx**: A modular framework for building custom DAOs with plugins (voting, token management, funding).
- **Aragon Court**: A decentralized dispute resolution system.

With Aragon, you can deploy a DAO in minutes, using templates for different governance models (e.g., token-based, multisig).

### 22.4.2 Snapshot

[Snapshot](https://snapshot.org/) is a off-chain voting platform. Votes are signed messages, not transactions, so they're free and gasless. Snapshot is widely used for signaling and governance in many DAOs (e.g., Uniswap, ENS).

**How it works:**
- Proposals are created on Snapshot's IPFS-backed interface.
- Token holders sign messages with their voting power (based on a snapshot of token balances at a certain block).
- Votes are tallied off-chain, and results are published.
- On-chain execution may still require a separate transaction (e.g., via a multisig or governor).

Snapshot supports various voting strategies (single choice, quadratic, ranked choice) and integrates with many token standards.

### 22.4.3 Governor Contracts (OpenZeppelin)

We've already used OpenZeppelin's Governor. It's the gold standard for on-chain governance, used by Compound, Uniswap, and many others. The contracts are highly modular and audited.

**Key components:**
- **Governor**: Base contract.
- **GovernorCountingSimple**: Simple for/against/abstain voting.
- **GovernorSettings**: Configurable voting delay, period, proposal threshold.
- **GovernorVotes**: Integrates with an ERC20Votes token.
- **GovernorVotesQuorumFraction**: Sets quorum as a percentage of total supply.
- **GovernorTimelockControl**: Adds timelock delay.

These can be mixed and matched to fit your needs.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 22 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  DAOs are organizations governed by code, not by central        │
│  authorities. They enable collective decision-making and        │
│  treasury management.                                           │
│                                                                 │
│  Governance mechanisms:                                         │
│    • Token-based voting (one token, one vote)                  │
│    • Quadratic voting (cost increases quadratically)           │
│    • Delegation (assign voting power to representatives)       │
│    • Proposal lifecycle: draft → pending → active → succeeded  │
│      → executed (or defeated)                                  │
│                                                                 │
│  Building a DAO with OpenZeppelin:                             │
│    • Governance token (ERC20Votes)                             │
│    • Governor contract (customizable settings)                 │
│    • Timelock (adds safety delay)                              │
│    • Integration via deployment scripts                         │
│                                                                 │
│  DAO tools:                                                    │
│    • Aragon: platform for creating DAOs (no-code & modular)    │
│    • Snapshot: off-chain voting (gasless)                      │
│    • OpenZeppelin Governor: on-chain governance standard       │
│                                                                 │
│  DAOs are still evolving—security, participation, and legal    │
│  frameworks are active areas of development.                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 23 – Ethereum Ecosystem Deep Dive. We'll explore Ethereum clients (Geth, Nethermind), the EIP process, the post-merge roadmap (Surge, Scourge, Verge, Purge), and important infrastructure like ENS.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='21. smart_contract_upgradeability.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../6. blockchain_networks_and_ecosystems/23. ethereum_ecosystem_deep_dive.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
